In [1]:
# importing dependencies
import pandas as pd
import numpy as np
import re 
import matplotlib.pyplot as plt
import seaborn as sns

from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# Importing evaluation metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [5]:
# printing the stopwords in english
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

### Data preprocessing

In [7]:
train = pd.read_csv(r"C:\Users\hp\Desktop\Data analysis\Python\Project\fake news\train.csv")

In [8]:
test = pd.read_csv(r"C:\Users\hp\Desktop\Data analysis\Python\Project\fake news\test.csv")

In [10]:
submit = pd.read_csv(r"C:\Users\hp\Desktop\Data analysis\Python\Project\fake news\submit.csv")

In [11]:
print(test.shape, train.shape, submit.shape)

(5200, 4) (20800, 5) (5200, 2)


In [12]:
train.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [13]:
# checking for missing data values
train.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [18]:
train.shape

(20800, 5)

In [24]:
# replacing null values with empty strings
train = train.fillna('') 

In [28]:
# merging the author name and news title together
train['content'] = train['author'] + ' ' + train['title']

In [30]:
train.head(1)

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...


In [31]:
# seperating lables and target
x = train.drop(columns='label', axis=1)
y = train['label']

In [32]:
# stemming procedure - process of reducing a word to its root word
port_stem = PorterStemmer()

In [38]:
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

def stemming(content):
    # Initialize a stemmer
    port_stem = PorterStemmer()
    
    # Replace non-alphabetic characters with a space
    stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
    
    # Convert to lowercase
    stemmed_content = stemmed_content.lower()
    
    # Tokenize the text into words
    stemmed_content = stemmed_content.split()
    
    # Perform stemming and remove stopwords
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if word not in stopwords.words('english')]
    
    # Join the stemmed words back into a single string
    stemmed_content = ' '.join(stemmed_content)
    
    return stemmed_content

def stemming(content):
    
    # replace all characters that are not letters (i.e., not in the range 'a' to 'z' and 'A' to 'Z') with a space in the content string. 
    # This step effectively removes any non-alphabetic characters from the text.
    stemmed_content = re.sub('[^a-zA-z],' ',content')
    
    # This line converts all the characters in stemmed_content to lowercase. 
    # This step ensures that the text is normalized and case-insensitive.
    stemmed_content = stemmed_content.lower()
    
    # This line splits the stemmed_content string into a list of words. 
    # By default, it splits the text by whitespace, effectively tokenizing the text into individual words.
    stemmed_content = stemmed_content.split()
    
    # This line performs stemming and removes stopwords from the list of words. 
    # It iterates through each word in the stemmed_content list, applies stemming using a stemming function
    # and checks if the word is not a stopword (using a list of English stopwords from stopwords.words('english')). 
    # If the word is not a stopword, the stemmed version of the word is added to a new list.
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    
    # This line joins the stemmed words back into a single string, with each word separated by a space.
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [39]:
train['content'] = train['content'].apply(stemming)

In [41]:
train.content[1]

'daniel j flynn flynn hillari clinton big woman campu breitbart'

In [42]:
# sepearting data into label and target
x = train['content'].values
y = train['label'].values

In [44]:
print(x)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']


In [45]:
print(y)

[1 0 1 ... 0 1 1]


In [46]:
print(x.shape, y.shape)

(20800,) (20800,)


In [47]:
# converting textual data to numerical data
vectorizer = TfidfVectorizer()

In [48]:
x = vectorizer.fit_transform(x)

In [51]:
#splitting x into test and train data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=2)

Training the model - Logistic Regression

In [52]:
model = LogisticRegression()

In [53]:
model.fit(x_train, y_train)

LogisticRegression()

In [54]:
train_pred = model.predict(x_train)
print(classification_report(y_train, train_pred))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99      8310
           1       0.98      0.99      0.99      8330

    accuracy                           0.99     16640
   macro avg       0.99      0.99      0.99     16640
weighted avg       0.99      0.99      0.99     16640



In [55]:
test_pred = model.predict(x_test)
print(classification_report(y_test, test_pred))

              precision    recall  f1-score   support

           0       0.99      0.96      0.98      2077
           1       0.97      0.99      0.98      2083

    accuracy                           0.98      4160
   macro avg       0.98      0.98      0.98      4160
weighted avg       0.98      0.98      0.98      4160



Making a prediction system

In [59]:
x_new = x_test[3]

prediction = model.predict(x_new)

if (prediction[0] == 0):
    print('The news is real')
else:
    print('The news is fake')

The news is real


In [60]:
print(y_test[3])

0
